# Import Statements

In [2]:
import OCR_pipeline as OCR
import TopicModeling as TM
import jsonpickle

[nltk_data] Downloading package punkt to /home/shivaram/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Text Extraction

In [6]:
#Next Step: Automatically extract relative paths
import storage
import pickle

textList = ["sample data/Kenya/AgricultureFisheriesandFoodAuthorityNo13of2013.PDF"]
PolicyPapers = {}
with open('policy_papers.pickle', 'rb') as readHandle:
    PolicyPapers = pickle.load(readHandle)

# Cleaning And Saving Extracted Text

In [8]:
#dictionary for texts in terms of sentences
import gc
textDictionary = {}
PolicyPapers.pop("sample data/Kenya/Sessional-paper-on-Kenya-National-Land-Policy.pdf", None)
PolicyPapers.pop("sample data/Kenya/Proposal_NFRA.pdf", None)
PolicyPapers.pop("sample data/Kenya/Kefristrategicplan2013-2018.pdf", None)
print(len(PolicyPapers))
for key in PolicyPapers:
    print(key)
    currentPaper = storage.deserializeObject(PolicyPapers[key])
    currentPaper.cleanedText = TM.toSentences(currentPaper.extractedText)
    storage.serializeObject(currentPaper)
    textDictionary[key] = currentPaper
   # gc.collect()

3
sample data/Kenya/Laboratory_Manual.pdf
sample data/Kenya/ClimateChangeAct11of2016 (3).pdf
sample data/Kenya/AgricultureFisheriesandFoodAuthorityNo13of2013.PDF


# LDA

In [9]:
from pprint import pprint
lemmatized_sents = []

# Print the Keywords in the 5 topics
for key in textDictionary:
    paper = textDictionary[key]
    lemmatized_sents.extend(paper.cleanedText)
    
corpus, lda_model = TM.LDA(lemmatized_sents)
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
#Next Step: Store LDA data in Policy Paper Object & other central file


[(19,
  '0.047*"former" + 0.040*"institution" + 0.038*"be" + 0.025*"liability" + '
  '0.021*"proceeding" + 0.020*"respect" + 0.018*"other" + 0.017*"protection" + '
  '0.017*"personal" + 0.017*"that"'),
 (17,
  '0.035*"be" + 0.022*"council" + 0.022*"may" + 0.018*"such" + '
  '0.018*"determine" + 0.017*"government" + 0.013*"agreement" + 0.013*"have" + '
  '0.011*"agriculture" + 0.011*"act"'),
 (1,
  '0.033*"land" + 0.027*"be" + 0.023*"may" + 0.019*"such" + 0.018*"require" + '
  '0.016*"appeal" + 0.016*"business" + 0.014*"order" + 0.014*"affair" + '
  '0.013*"person"'),
 (20,
  '0.048*"be" + 0.023*"institution" + 0.020*"carry" + 0.018*"commercial" + '
  '0.017*"shall" + 0.016*"activity" + 0.016*"compliance" + 0.016*"former" + '
  '0.015*"act" + 0.014*"monitor"'),
 (28,
  '0.035*"-PRON-" + 0.032*"act" + 0.026*"shall" + 0.024*"other" + '
  '0.017*"authority" + 0.016*"person" + 0.015*"contract" + '
  '0.015*"consideration" + 0.015*"interest" + 0.015*"law"'),
 (24,
  '0.033*"be" + 0.031*"siev

# Storage